In [1]:
import pandas as pd
from datetime import datetime
import json
import tweepy
from parse_raw_tweet import parse_raw_tweet
import base64
import yaml
import time
from getTopic import handleData
from getTopic import df2tweet
from tqdm.auto import tqdm

from Helper import Helper
from Tweet import Tweet
import re

# CONSUMER_KEY = st.secrets['CONSUMER_KEY']
# CONSUMER_SECRET = st.secrets['CONSUMER_SECRET']
# ACCESS_TOKEN = st.secrets['ACCESS_TOKEN']
# ACCESS_SECRET = st.secrets['ACCESS_SECRET']

CONSUMER_KEY = 'NklB4SGVsSjpXLYWVn6lXxd7U'
CONSUMER_SECRET = 'rUMAU5dIpIRTImwIBNfgxxYWwP7Y2pkgjB3X36J7JkjkfLIkMu'
ACCESS_TOKEN = '1440879620971184129-HyGEUpcpEdHedXusFfaxRZ0NlyD1hA' #Authorization: Bearer
ACCESS_SECRET = 'OIwnv2pjyT9qjgFU1PnpUIbSSWittZFnoGMojUDNAz58B'
# Bearer: AAAAAAAAAAAAAAAAAAAAADU8UAEAAAAAGVXvwcwh%2BgCv%2FFhm7rDZgdmUv4k%3DeCwAbKOPDfhNcYENvdv4B8B9nGTR2Xje6whEqCOPEI8bpks4vn
def read_config():

	## authenticate
	auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
	auth.set_access_token(ACCESS_TOKEN, ACCESS_SECRET)
	api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True, compression=True)


	return api

C:\Users\ngu00336\AppData\Roaming\Python\Python37\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ngu00336\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\ngu00336\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ngu00336\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
topics_cols=[]
with open("data/keywords.json", "r") as read_file:
    topics_cols =  list(json.load(read_file))

def addValue(df2,i,keyIndex,result):
    c = result[i][0]['top'+str(keyIndex)].values[0]
    df2.at[i,c] = result[i][0]['score'+str(keyIndex)].values[0]
    df2.at[i,c+"_keywords"] = str(result[i][1][c+'_keywords'].values[0])

In [3]:
def savetweet(df2,result,fileName):
    for c in topics_cols:
        df2[c]=0
        df2[c+"_keywords"]='()'
    for i in tqdm(range(0,df2.shape[0])):
        addValue(df2,i,1,result)
        addValue(df2,i,2,result)
        addValue(df2,i,3,result)
        addValue(df2,i,4,result)
        addValue(df2,i,5,result)
    df2.to_csv('data/real_10_14_22_2021/'+fileName+'_topic.csv',index=False)

In [4]:
api = read_config()

In [5]:
startDate = datetime(2021, 10, 14, 0, 0, 0)
endDate =   datetime(2021, 10, 21, 0, 0, 0)

In [6]:
def getFromsource(username):
    tweets = []
    tmpTweets = api.user_timeline(username)
    
    
    for tweet in tmpTweets:
        if tweet.created_at < endDate and tweet.created_at > startDate:
            tweets.append(tweet)
    result = []
    df2 = pd.DataFrame(columns=['tweetid','userid','user_display_name','tweet_time'])
    while (tmpTweets[-1].created_at > startDate):
        tmpTweets = api.user_timeline(username, max_id = tmpTweets[-1].id)
        for tweet in tmpTweets:
            if tweet.created_at < endDate and tweet.created_at > startDate:
                raw_tweet = api.get_status(tweet.id, tweet_mode="extended")._json
                [top_n_topics_df,keywords_appearing_df] = handleData(raw_tweet)
                result.append([top_n_topics_df,keywords_appearing_df])
                df = {'tweetid':raw_tweet['id'],'userid':raw_tweet['user']['id'],'user_display_name':raw_tweet['user']['name'],
                      'tweet_time':raw_tweet['created_at']}
                df2 = df2.append(df, ignore_index = True)
    savetweet(df2,result,username)

In [7]:
listNews = pd.read_excel("data/News List and dictionary.xlsx",sheet_name='News list')
listNews = listNews[pd.isna(listNews["Type"])]

In [ ]:
for i in tqdm(range(0,listNews.shape[0])):
    try:
        news = listNews.iloc[i]
        user_name = news['Twitter Link'].split('/')[-1]
        print(i,user_name)
        getFromsource(user_name)
    except:
        print('faile')

104 sacbee_news
expandURL(link) https://trib.al/HfH3Umv
faile
105 ocregister
faile
106 EastBayTimes
faile
107 sfchronicle
faile
108 FresnoBee
faile
109 sdut
faile
110 pressenterprise
faile
111 sfexaminer
faile
112 SGVTribune
faile
113 DailyBreezeNews
faile
114 modbee
faile
115 ladailynews
faile
116 NorthBayNews
faile
117 presstelegram
faile
118 ivdailybulletin
faile
119 SLOTribune
faile
120 MyDesert
faile
121 BreakingNews_RS
faile
122 ChicoER
faile
123 smdailypress
faile
124 sbnpnews
faile
125 MontereyHerald
faile
126 eurekaTS
faile
127 Bakersfieldcali
faile
128 MercedSunStar
faile
129 TheMendoVoice
faile
130 appealdemocrat
faile
131 NapaRegister
faile
132 redbluffnews
faile
133 RedlandsNews
faile
134 salnews
faile
135 PasStarNews
faile
136 recorderonline
faile
137 smdailyjournal
faile
138 SCVSignal
faile
139 scsentinel
faile
140 vcstar
faile
141 woodlandnews
faile
142 WhittierNews
faile
143 EastBayExpress
faile
144 thepress_net
faile
145 Alameda_Sun
faile
146 TimesAdvocate
faile
147 h